[![image.png](./images/QA_Chain.png)]

In [1]:

from langchain.vectorstores import Chroma
from langchain_ollama import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="llama3.2:3b",
)

persist_directory = 'docs/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

C:\Users\Yazat\AppData\Local\Temp\ipykernel_15596\4208993829.py:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)


In [2]:
print(vectordb._collection.count())

8


In [4]:
question = "What Yazat's major skills"
docs = vectordb.similarity_search(question,k=3)

len(docs)

3

In [5]:
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model='llama3.2:3b', temperature=0)


In [6]:
from langchain.chains import RetrievalQA

In [7]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [8]:
result = qa_chain({"query": question})

C:\Users\Yazat\AppData\Local\Temp\ipykernel_15596\4094420968.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": question})


In [9]:
result["result"]

"Based on the provided context, Yazat Mishra's major skills include:\n\n1. Programming languages: C/C++, Python, JavaScript, TypeScript, HTML/CSS.\n2. Frameworks: ReactJS, Express, NodeJS, Jest.\n3. Libraries: TensorFlow, Keras, Torch, Pandas, NumPy, Matplotlib, Scikit-Learn, GDAL, seaborn.\n\nThese skills are mentioned in Yazat's internship experience, projects, and technical toolkit section."

In [10]:
from langchain.prompts import PromptTemplate

# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [11]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [12]:
question = "Is Yazat an undergraduate?"

In [13]:
result = qa_chain({"query": question})

In [14]:
result["result"]

'Yazat is a final-year B.Tech student, indicating that they are still an undergraduate. Thanks for asking!'

In [15]:
result["source_documents"][0]

Document(metadata={'page': 0, 'source': 'docs/RANK Certificate_Yazat Mishra.pdf'}, page_content='Internship.  \n \n…………………………………………………… \n    Associate Dean (SW) and FIC, T&P \nNational Institute of Technology Mizoram \nEmail: adean_sw@nitmz.ac.in \n \nThis certificate has been issued with the approval of the Director, National Institute of Technology Mizoram')

In [17]:
qa_chain_mr = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    chain_type="refine"
)
result = qa_chain_mr({"query": question})
result["result"]

'Based on the provided context information and additional details from Yazat\'s resume, it is now possible to determine that Yazat is likely an alumnus of their university.\n\nThe text mentions that Yazat is a "final-year B.Tech student" in Electronics and Communication at the National Institute of Technology Mizoram, which implies that they are likely an undergraduate. However, considering the context provided by Yazat\'s resume, it appears that they have already completed their final year of studies, as they mention being a "final-year B.Tech student" and also having spearheaded several projects.\n\nFurthermore, Yazat has been involved in various extracurricular activities such as Placement Coordinator and Technical Head of the ECE Society at NIT Mizoram, which suggests that they have graduated from undergraduate studies and are now in the process of transitioning to a professional environment.\n\nTherefore, while it was initially unclear whether Yazat is an undergraduate or not base